# Transformers para Martin Fierro

Los dos principales objetivos de este tutorial son mostrar cómo es una arquitectura transformer y cómo usar las herramientas provistas por [Hugging Face](https://huggingface.co/). En este tutorial usaremos una implementación de [GPT2](https://openai.com/blog/better-language-models/) en español para generar (una vez más) texto similar al Martín Fierro, que se puede descargar [aquí](https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/martin_fierro.txt).

Créditos a [Jay Alammar](https://jalammar.github.io/illustrated-gpt2/) por las imágenes.

In [279]:
import torch
import numpy as np
import pandas as pd
import transformers
import torch.nn as nn
import os
import sys
from time import time
import wget
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import unicodedata, re

use_cuda = torch.cuda.is_available()

## GPT2

GPT2 es un modelo de generación de texto basado en transformers. A diferencia de BERT u otros transformers, GPT2 está basado en sucesivas instancias de decoders

<img src="images/gpt-2-transformer-xl-bert-3.png"
     alt="GPT2 Decoder Architecture"
     style="float: center; margin-right: 150px;"
     width=75%/>
     
<img src="images/gpt2_function.gif"
     alt="GPT2 Text Generation Function"
     style="float: center; margin-right: 150px;"
     width=75%/>
     
**Recursos:**

[The Illustrated GPT-2 (Visualizing Transformer Language Models)](https://jalammar.github.io/illustrated-gpt2/)

## Parte 1: Usando transformers de Hugging Face

La librería transformers permite usar modelos ya entrenados en un pipeline y nos facilita funciones que procesan el input, lo tokenizan y hace el forward pass para generar las predicciones.

Dada una tarea, pipeline descarga un modelo y tokenizador apropiados para la tarea. En este caso, especificamos un modelo preentrenado de GPT2-small en idioma español.

<img src="images/gpt2_sizes.png"
     alt="GPT2 Model Sizes"
     style="float: center; margin-right: 200px;"
     width=70%/>
     
<img src="images/gpt2_decoder_stacks.png"
     alt="GPT2 Model Sizes"
     style="float: center; margin-right: 200px;"
     width=70%/>
     

Para nuestra tarea en particular, pipeline tomará un texto e internamente se encargará del tokenizado de la oración y de generar texto hasta el *max_length* indicado.

In [209]:
from transformers import pipeline

generator = pipeline('text-generation', model="datificate/gpt2-small-spanish")

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at datificate/gpt2-small-spanish.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at datificate/gpt2-small-spanish.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [254]:
input_sentence = 'Ahí dice: "los miminos bien"'
output = generator(input_sentence, max_length=100)
print('Generated Text: %s' % input_sentence)
print('... ', output[0]['generated_text'][len(input_sentence):])

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Generated Text: Ahí dice: "los miminos bien"
...   y el de "los tritones son muy dolorosos".


El rey Arturo, hijo de la princesa Olvi (María de Molina), se opone al matrimonio: él quiere que nadie los consuma como reina, pues considera que lo que la ha hecho feliz, para él, es el matrimonio con una mujer en la forma de mujer. Arturo, a su vez, no ha sido un candidato para alcanzar a la corona, pero se


Para usar otras tareas de 🤗 Transformers, [aquí](https://huggingface.co/transformers/task_summary.html) hay una lista completa de todas las tareas que se implementan.

---

## Parte 1: Importando de 🤗 Transformers

Lo primero es importar el modelo con el cuál trabajaremos. En nuestro caso usaremos [GPT2-Small en español de Datificate](https://huggingface.co/datificate/gpt2-small-spanish) (pueden jugar con el modelo desde la misma página de 🤗). 

Nuestrea tarea de interés es hacer Language Modeling del Martín Fierro, por lo cual importamos **GPT2LMHeadModel**, que es una implementación de GPT2 con la última capa lineal para retornar los logits del tamaño del vocabulario.

Como el modelo *datificate/gpt2-small-spanish* fué entrenado en tensorflow, usamos el flag *from_tf=True* para asegurarnos que importe los pesos correctamente.

In [6]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('datificate/gpt2-small-spanish', from_tf=True)

All TF 2.0 model weights were used when initializing GPT2LMHeadModel.

Some weights of GPT2LMHeadModel were not initialized from the TF 2.0 model and are newly initialized: ['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAI

El paso siguiente es descargar un tokenizador. El tokenizador hará por nosotres el trabajo de separar el texto en palabras y convertirlas en sus correspondientes ids dentro del vocabulario. Al igual que con el modelo, 🤗 provee tokenizers ya implementados para sus correspondientes modelos.

In [230]:
tokenizer = GPT2TokenizerFast.from_pretrained('datificate/gpt2-small-spanish', add_prefix_space=True)

tokenized_seq = tokenizer("Yo nunca habia escuchado hablar de Innsmouth hasta")
print('inputs_ids: {}'.format(tokenized_seq['input_ids']))
print('attention_mask: {}'.format(tokenized_seq['attention_mask']))
tokens = tokenizer.convert_ids_to_tokens(tokenized_seq['input_ids'])
print('tokens: {}'.format(tokens))

inputs_ids: [4701, 2686, 498, 363, 27422, 6242, 258, 599, 10007, 15308, 657]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tokens: ['ĠYo', 'Ġnunca', 'Ġhab', 'ia', 'Ġescuchado', 'Ġhablar', 'Ġde', 'ĠIn', 'ns', 'mouth', 'Ġhasta']


El tokenizer se puede encargar por nosotres de manejar temas de padding y truncado entre otros. Para más info en las funciones del tokenizers, mirar [aquí](https://huggingface.co/transformers/main_classes/tokenizer.html).

El tokenizer también puede procesar múltiples oraciones al mismo tiempo y retornar tensores para el framework que estemos usando de base, sea este TensorFlow 2.0 ('*tf*') o PyTorch ('*pt*').

In [239]:
sentences = ["Yo nunca habia escuchado hablar de Innsmouth hasta",
             "Hacia el oeste de Arkham las montañas se levantaban indómitas y en sus entrañas"]

In [240]:
tokenized_seq = tokenizer(sentences,
                          padding=True,
                          return_tensors="pt")

print('inputs_ids:\n{}'.format(tokenized_seq['input_ids']))
print('attention_mask:\n{}'.format(tokenized_seq['attention_mask']))

inputs_ids:
tensor([[ 4701,  2686,   498,   363, 27422,  6242,   258,   599, 10007, 15308,
           657,     0,     0,     0,     0,     0,     0,     0,     0],
        [11945,   284,  2866,   258, 18307,  2747,   347,  6674,   306,  3620,
          4413,  1582,  1178,  2982,   287,   278,   452,  7259,  1846]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


Incluso el tokenizer puede manejar listas de palabras con solo agregar el parámetro *is_split_into_words=True*.

In [242]:
tokenized_seq = tokenizer([sent.split() for sent in sentences],
                          padding=True,
                          return_tensors="pt",
                          is_split_into_words=True)

print('inputs_ids:\n{}'.format(tokenized_seq['input_ids']))
print('attention_mask:\n{}'.format(tokenized_seq['attention_mask']))

inputs_ids:
tensor([[ 4701,  2686,   498,   363, 27422,  6242,   258,   599, 10007, 15308,
           657,     0,     0,     0,     0,     0,     0,     0,     0],
        [11945,   284,  2866,   258, 18307,  2747,   347,  6674,   306,  3620,
          4413,  1582,  1178,  2982,   287,   278,   452,  7259,  1846]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


---
## Parte 2: Finetune con PyTorch nativo

Para entrenar con PyTorch, necesitamos crear una instancia de un Dataset de PyTorch. Esta vez, podemos usar el tokenizer que ya importamos para hacer el encoding de las palabras.

Por simplicidad, dividiremos el el conjunto de datos en palabras y usaremos listas de palabras de tamaño fijo.

Para entrenamiento nativo en Tensorflow 2, ver [aquí](https://huggingface.co/transformers/training.html#fine-tuning-in-native-tensorflow-2).

In [257]:
from torch.utils.data import Dataset

class MartinFierroDatasetGPT2(Dataset):
    def __init__(self, textdata, maxlen, tokenizer):
        
        self.maxlen = maxlen
        # Get ourselves a list of words so we can iterate
        split_text = textdata.split()
        # cut the text in sequences of maxlen characters
        self.sentences = {}
        for idx, i in enumerate(range(0, len(split_text) - maxlen - 1, maxlen)):
            self.sentences[idx] = split_text[i: i + maxlen]

        # You need to activate padding in order to
        # return tensors
        self.data = tokenizer(list(self.sentences.values()),
                              padding=True,
                              is_split_into_words=True,
                              return_tensors="pt")
        
        self.length = len(self.sentences)
        print('NB sequences:', self.length)
        
    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}
        return item


In [258]:
with open('./martin_fierro.txt', 'r') as finput:
    text = unicodedata.normalize('NFC', finput.read()).lower()
    text = re.sub('\s+', ' ', text).strip()

print('Corpus length: %d' % len(text))

train_dataset = MartinFierroDatasetGPT2(text, 50, tokenizer)

Corpus length: 33858
NB sequences: 128


In [293]:
model.train()
device = torch.device('cuda') if use_cuda else torch.device('cpu')

model.to(device)
loss_function = nn.NLLLoss()


In [294]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

In [295]:
from torch.utils.data import DataLoader

dataloader_config = {'dataset': train_dataset,
                     'batch_size': 1,
                     'shuffle': True,
                     'num_workers': 0,
                     'pin_memory': use_cuda}

dataloader = DataLoader(**dataloader_config)

In [ ]:
from tqdm.notebook import tqdm

stream = tqdm(enumerate(dataloader), total=len(dataloader))
for i, sample in stream:
    start = time()
    input_ids = sample['input_ids'].to(device)
    attention_mask = sample['attention_mask'].to(device)
    
    outputs = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=input_ids)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    stream.set_description('loss=%g, elapsed=%g' % (loss, time()-start))

## Parte 3: Entrenando con Transformers Trainers

In [288]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm=False)

training_args = TrainingArguments(
    output_dir='./gpt2-fierro',      # output directory
    num_train_epochs=1,              # total # of training epochs
    per_device_train_batch_size=1 ,  # batch size per device during training
    per_device_eval_batch_size=1,    # batch size for evaluation
    warmup_steps=1,                  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                     # the instantiated 🤗 Transformers model to be trained
    args=training_args,              # training arguments, defined above
    train_dataset=train_dataset,     # training dataset (THE SAME AS BEFORE)
    data_collator=data_collator
)

In [289]:
trainer.train()

/users/mmazuecos/anaconda3/envs/visdial/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/users/mmazuecos/anaconda3/envs/visdial/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


TrainOutput(global_step=43, training_loss=5.386078945426053, metrics={'train_runtime': 29.5643, 'train_samples_per_second': 1.454, 'total_flos': 8314570211328, 'epoch': 1.0})

## Parte 4: Usando nuestro nuevo modelo en un pipeline